### Projeto Final de Spark - Avançado
#### Campanha Nacional de Vacinação contra Covid-19

Autor: Arthur Freitas Gomes

Turma: Semantix Academy - Big Data Engineer 04-21

Professor: Rodrigo Rebouças

##### Nível Avançado:

Replicar as visualizações do site “https://covid.saude.gov.br/”, porém acessando diretamente a API de Elastic.

Link oficial para todas as informações:  https://opendatasus.saude.gov.br/dataset/covid-19-vacinacao

Informações para se conectar ao cluster:

    > URL https://imunizacao-es.saude.gov.br/desc-imunizacao
    > Nome do índice: desc-imunizacao
    > Credenciais de acesso
    > Usuário: imunizacao_public
    > Senha: qlto5t&7r_@+#Tlstigi

In [118]:
import requests
import pandas as pd

In [119]:
#Acesso inicial para gravar '_scroll_id'
covid_API = requests.get('https://imunizacao-es.saude.gov.br/_search?scroll=1m',
                         auth=requests.auth.HTTPBasicAuth('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'),
                         params={"size":10000})

In [130]:
#covid_API.json()

In [127]:
#'_scroll_id' = id temporário que será utilizado para navegar nos scrolls com 10000 registros cada
covid_API.json()['_scroll_id']

'FGluY2x1ZGVfY29udGV4dF91dWlkDnF1ZXJ5VGhlbkZldGNoAxZNS1hZTVE5c1RFV1d5TnBQdV94RHVnAAAAAG06vuAWVGNTY2RHVmdRTUtxVjBiQXFFMEJCURZacERfeXZFNlI4eUpObkU0RGFMMEtRAAAAAG0nMRQWOE1zR2RFMGFSaGl2Zndtc0dnMW9NURY3ZGlmdS1PcFNXbTBxTUhIbWhpdnB3AAAAAGtxAqsWU1BFYmVBMDBRV1dULWt4VDJxczEwUQ=='

In [128]:
#Número de registros por Scroll
len(covid_API.json()['hits']['hits'])

10000

In [123]:
#Scroll = 0 

count_scroll = 0

scroll_id = covid_API.json()['_scroll_id']
covid_API_scroll = requests.get('https://imunizacao-es.saude.gov.br/_search/scroll',
                                auth=requests.auth.HTTPBasicAuth('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'),
                                params={"scroll_id":scroll_id, "scroll":'1m'})

#recebe infomações HITS, seleciona dict _source e salva em um json
df_loc = pd.DataFrame(covid_API_scroll.json()['hits']['hits'])
df_loc['_source'].to_json('api_covid/covid_scroll_'+str(count_scroll)+'.json')

In [131]:
#Loop para repetir processo anterior até hit  = 0

"""
Cada arquivo .json está com ~15Mb e temos 116.089.044 hits, ou seja, 11.608 arquivos, para o exemplo limitei a 10 arquivos
"""

while count_scroll < 10:
#while len(covid_API_scroll.json()['hits']['hits']) != 0:
    scroll_id = covid_API.json()['_scroll_id']
    covid_API_scroll = requests.get('https://imunizacao-es.saude.gov.br/_search/scroll', auth=requests.auth.HTTPBasicAuth('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'), params={"scroll_id":scroll_id, "scroll":'1m'})

    #covid_API_scroll.json()
    #covid_API_scroll.json()['hits']['hits']
    
    count_scroll += 1
    df_loc = pd.DataFrame(covid_API_scroll.json()['hits']['hits'])
    df_loc['_source'].to_json('api_covid/covid_scroll_'+str(count_scroll)+'.json')


In [132]:
!ls api_covid/

covid_scroll_0.json   covid_scroll_3.json  covid_scroll_7.json
covid_scroll_1.json   covid_scroll_4.json  covid_scroll_8.json
covid_scroll_10.json  covid_scroll_5.json  covid_scroll_9.json
covid_scroll_2.json   covid_scroll_6.json


In [133]:
pd.read_json('api_covid/covid_scroll_0.json').T.head()

,estabelecimento_razaoSocial,vacina_dataAplicacao,vacina_grupoAtendimento_codigo,estabelecimento_valor,@timestamp,sistema_origem,vacina_lote,id_sistema_origem,estalecimento_noFantasia,paciente_endereco_coIbgeMunicipio,...,vacina_descricao_dose,vacina_fabricante_nome,vacina_categoria_codigo,paciente_endereco_uf,vacina_categoria_nome,redshift,vacina_nome,paciente_racaCor_valor,paciente_id,paciente_enumSexoBiologico
0,PREFEITURA MUNICIPAL DE SANTO ANDRE,2021-02-16T00:00:00.000Z,000926,0008699,2021-07-09T15:56:02.556Z,VACIVIDA,202010029,18262,UBS JARDIM IRENE,355030,...,2ª Dose,Sinovac,9,SP,Trabalhadores de Saúde,v2,Covid-19-Coronavac-Sinovac/Butantan,SEM INFORMACAO,464ace8c056e3eb67ebdc8ad0a198b74f259d11700003b...,F
1,MUNICIPIO DE ITAJAI,2021-02-08T00:00:00.000Z,000907,2513870,2021-07-09T15:56:02.557Z,G-MUS,4120Z005,17901,CRESCEM CENTRO DE REFERENCIA DA SAUDE DA CRIAN...,420910,...,1ª Dose,FUNDACAO OSWALDO CRUZ,9,SC,Trabalhadores de Saúde,v2,Vacina Covid-19 - Covishield,BRANCA,2890fdfe35dd36eac48539c3740463bb4862d92e3f86cb...,M
2,SECRETARIA MUNICIPAL DE SAUDE,2021-01-22T00:00:00.000Z,000926,6449409,2021-07-09T15:56:02.225Z,Novo PNI,202010031,16341,SECRETARIA MUNICIPAL DE SAUDE DE GOIANIA,330455,...,1ª Dose,FUNDACAO BUTANTAN,9,RJ,Trabalhadores de Saúde,v2,Covid-19-Coronavac-Sinovac/Butantan,SEM INFORMACAO,889cef0b8fbe6cc8f2cf4057fdf5cd7b8bb5b87457c89b...,F
3,IRMANDADE DA SANTA CASA DE MISERICORDIA DE MAUA,2021-02-05T00:00:00.000Z,000926,2751747,2021-07-09T15:56:02.225Z,VACIVIDA,202010029,18262,SANTA CASA DE MAUA,352940,...,1ª Dose,Sinovac,9,SP,Trabalhadores de Saúde,v2,Covid-19-Coronavac-Sinovac/Butantan,PARDA,6e92378d3c4fa812b264d2d917cd97c4f3e1040b663991...,F
4,MUNICIPIO DE ITAJAI,2021-02-10T00:00:00.000Z,000801,6539858,2021-07-09T15:56:02.225Z,G-MUS,4120Z005,17901,SECRETARIA MUNICIPAL DE SAUDE DE ITAJAI,420820,...,1ª Dose,FUNDACAO OSWALDO CRUZ,8,SC,Trabalhadores da Educação,v2,Vacina Covid-19 - Covishield,PARDA,8236dd899effef95aaf2f73e18fc59a437f3028c8dd916...,M


##### Próximos passos:

    > enviar para namenode;
    > criar visualizações.